# Libraries

In [1]:
%pip install -q -U datasets transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
import os
import random
import numpy as np
import torch
import transformers
from pprint import pprint
from datetime import datetime
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, Trainer, TrainingArguments
from sklearn.metrics import f1_score

# Config

In [3]:
seed = 42
lang = 'sun'

# hf_model_avg_id = 'FacebookAI/roberta-base'
# hf_model_avg_id = 'FacebookAI/xlm-roberta-base'
hf_model_avg_id = 'w11wo/sundanese-roberta-base-emotion-classifier'
# hf_model_ids = [
#     'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222031929',
#     'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222033713',
#     'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222034933',
#     'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222041052',
# ]
# hf_model_ids = [
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241222063252',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241222070822',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241222071909',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241222072806',
# ]
hf_model_ids = [
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222094914',
    'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222100050',
    # 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222101518',
    # 'alxxtexxr/RoBERTa-Base-SE2025T11A-sun-v20241222102826',
]
# hf_model_ids = [
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241222174522',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241223001348',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241223002908',
#     'alxxtexxr/XLM-RoBERTa-Base-SE2025T11A-sun-v20241223004034',
# ]
hf_tokenizer_id = hf_model_avg_id
hf_data_id = 'alxxtexxr/SemEval2025-Task11-Dataset'
hf_data_config = 'track_a_sun_70_15_15_stratify_v2'

project_name = f'RoBERTa-Base-Avg-SE2025T11A-{lang}-v{datetime.now().strftime("%Y%m%d%H%M%S")}'

In [4]:
def set_seed(seed):
    # Set random seed for NumPy
    np.random.seed(seed)

    # Set random seed for Torch
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensures deterministic results
    torch.backends.cudnn.benchmark = False  # Avoids non-deterministic algorithms

    # Set random seed for Transformers
    transformers.set_seed(seed)

    # Optionally set random seed for sklearn and Python's own random module
    random.seed(seed)

    # Set random seed for os
    os.environ['PYTHONHASHSEED'] = str(seed)

    print(f"Random seed set to: {seed}")

set_seed(seed)

Random seed set to: 42


# Data

## Load Data

In [5]:
datasets = load_dataset(hf_data_id, hf_data_config)

cols = list(datasets['train'].features)
emotion_cols = [col for col in cols if col not in ['Unnamed: 0', 'text', 'emotion']]
splits = [*datasets.keys()]

print("Data columns:", cols)
print("Emotions columns:", emotion_cols)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md:   0%|          | 0.00/1.86k [00:00<?, ?B/s]

(…)ack_a/sun_70_15_15_stratify_v2/train.csv:   0%|          | 0.00/68.1k [00:00<?, ?B/s]

(…)track_a/sun_70_15_15_stratify_v2/val.csv:   0%|          | 0.00/13.3k [00:00<?, ?B/s]

(…)rack_a/sun_70_15_15_stratify_v2/test.csv:   0%|          | 0.00/12.9k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Data columns: ['text', 'emotion', 'marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']
Emotions columns: ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [6]:
class2id = {class_:id for id, class_ in enumerate(emotion_cols)}
id2class = {id:class_ for class_, id in class2id.items()}

print("Class to ID:\n")
pprint(class2id, width=1)
print()
print("ID to Class:\n")
pprint(id2class, width=1)

Class to ID:

{'biasa': 6,
 'jijik': 1,
 'marah': 0,
 'sedih': 4,
 'senang': 3,
 'takut': 2,
 'terkejut': 5}

ID to Class:

{0: 'marah',
 1: 'jijik',
 2: 'takut',
 3: 'senang',
 4: 'sedih',
 5: 'terkejut',
 6: 'biasa'}


## Preprocess Data

In [7]:
tokenizer = AutoTokenizer.from_pretrained(hf_tokenizer_id)

tokenizer_config.json:   0%|          | 0.00/318 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/786k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/445k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.33M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
def one_hot_encode_emotion(emotion, emotion_cols):
    emotions = emotion.replace(" ", "").split(",")
    one_hot_emotion = [1.0 if emotion_col in emotions else 0.0 for emotion_col in emotion_cols] # Ensure that the label is float, not int
    return one_hot_emotion

def preprocess_function(data):
   text = data['text']
   emotion = data['emotion']
   labels = one_hot_encode_emotion(emotion, emotion_cols)
   data = tokenizer(text, truncation=True)
   data['labels'] = labels
   return data

tokenized_datasets = {split: datasets[split].map(preprocess_function) for split in splits}

Map:   0%|          | 0/668 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/128 [00:00<?, ? examples/s]

Map:   0%|          | 0/128 [00:00<?, ? examples/s]

In [9]:
# Sanity check
data = tokenized_datasets['train'][5]

print("Text:", data['text'])
print("Emotion(s):", data['emotion'])
print("Labels:", data['labels'], '-->', emotion_cols)

Text: ☝️Cung urang Kuningan, Pas urang waduk Darma, Kpan atuh main ka waduk Darma Dai, ??? Di antos, Aa sareng tteh Na, 🤗🤗🤗
Emotion(s): senang
Labels: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0] --> ['marah', 'jijik', 'takut', 'senang', 'sedih', 'terkejut', 'biasa']


In [10]:
data_collator = DataCollatorWithPadding(tokenizer)

# Model

In [11]:
models = []
for hf_model_id in hf_model_ids:
    model = AutoModelForSequenceClassification.from_pretrained(
        hf_model_id, 
        num_labels=len(emotion_cols),
        id2label=id2class, 
        label2id=class2id,
        problem_type = "multi_label_classification",
    )
    models.append(model)
    
# print("Models:", models)

config.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/978 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [12]:
model_avg = AutoModelForSequenceClassification.from_pretrained(
    hf_model_avg_id, 
    # num_labels=len(emotion_cols),
    # id2label=id2class, 
    # label2id=class2id,
    problem_type = "multi_label_classification",
)

config.json:   0%|          | 0.00/942 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [13]:
model_avg.classifier.out_proj = torch.nn.Linear(in_features=768, out_features=len(emotion_cols), bias=True)
print(model_avg.classifier)

RobertaClassificationHead(
  (dense): Linear(in_features=768, out_features=768, bias=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (out_proj): Linear(in_features=768, out_features=7, bias=True)
)


## Model Averaging

In [14]:
with torch.no_grad():
    for param_avg in model_avg.state_dict().keys():
        weights = [model.state_dict()[param_avg] for model in models]
        weight_avg = sum(weights) / len(weights)
        model_avg.state_dict()[param_avg].copy_(weight_avg)

# Evaluation

In [23]:
def sigmoid(x):
   return 1/(1 + np.exp(-x))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    probs = sigmoid(predictions)
    y_pred = (probs > 0.5).astype(int)
    y_true = labels.astype(int)

    # Compute F1 score for each type of averaging method
    f1_micro = f1_score(y_true, y_pred, average='micro', zero_division=0.0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0.0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0.0)
    f1_samples = f1_score(y_true, y_pred, average='samples', zero_division=0.0)
    f1_labels = f1_score(y_true, y_pred, average=None, zero_division=0.0)
    f1_labels_dict = {f'f1_label_{emotion_cols[i]}': f1_labels[i] for i in range(len(f1_labels))}

    return {
        'f1_macro': f1_macro,
        'f1_micro': f1_micro,
        'f1_weighted': f1_weighted,
        'f1_samples': f1_samples,
        **f1_labels_dict,
    }

In [21]:
train_args = TrainingArguments(
    # Training config
    per_device_train_batch_size=2,
    num_train_epochs=8,
    learning_rate=2e-5,
    weight_decay=0.01,

    # Logging config for training
    logging_strategy='steps',
    logging_steps=100,

    # Evaluation config during training
    per_device_eval_batch_size=2,
    eval_strategy='steps',
    eval_steps=100,

    # Model saving config
    output_dir=project_name,
    save_strategy='epoch',
    # load_best_model_at_end=True,
)

trainer = Trainer(
    model=model_avg,
    args=train_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

<ipython-input-21-66cc8af75179>:23: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
eval = trainer.evaluate(eval_dataset=tokenized_datasets['test'])

import pandas as pd
pd.DataFrame(eval, index=[0])

,eval_loss,eval_model_preparation_time,eval_f1_weighted,eval_f1_samples,eval_f1_micro,eval_f1_macro,eval_f1_label_marah,eval_f1_label_jijik,eval_f1_label_takut,eval_f1_label_senang,eval_f1_label_sedih,eval_f1_label_terkejut,eval_f1_label_biasa,eval_runtime,eval_samples_per_second,eval_steps_per_second
0,0.361826,0.0032,0.671834,0.720052,0.693252,0.354305,0.153846,0.222222,0.0,0.876289,0.666667,0.311111,0.25,0.7066,181.157,90.578


In [18]:
trainer.push_to_hub(project_name)

events.out.tfevents.1735037240.b33206213a8c.2617.0:   0%|          | 0.00/847 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.37k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241224104615/commit/8c723a77d7329f0ae247ad79c004e0108caedaea', commit_message='RoBERTa-Base-Avg-SE2025T11A-sun-v20241224104615', commit_description='', oid='8c723a77d7329f0ae247ad79c004e0108caedaea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241224104615', endpoint='https://huggingface.co', repo_type='model', repo_id='alxxtexxr/RoBERTa-Base-Avg-SE2025T11A-sun-v20241224104615'), pr_revision=None, pr_num=None)

# Inference

In [19]:
data = datasets['val'][1]
text = data['text']
emotion_true = data['emotion']

inputs = tokenizer(text, return_tensors='pt').to(model_avg.device)

outputs = trainer.model(**inputs)
logits = outputs.logits
probs = sigmoid(logits.squeeze().detach().cpu().numpy()) # apply sigmoid + threshold
labels_pred = (probs > 0.5).astype(int)
emotion_pred = [id2class[idx] for idx, label in enumerate(labels_pred) if label == 1.0] # turn predicted id's into actual label names

print("Text:", text)
print("True emotion(s):", emotion_true)
print("Predicted emotion(s):", ", ".join(emotion_pred))

Text: Nongton iklan nepi ka tamat ,😭
True emotion(s): senang
Predicted emotion(s): sedih
